In [3]:
import time
import logging
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import os

In [4]:
#### 1. Import modules

from selenium import webdriver
#from selenium.webdriver.chrome.service import Service
from selenium.webdriver.firefox.options import Options # 1. import modulesns
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
import pandas as pd
import os

#### 2. Define the Gecko driver



In [5]:
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'

options.add_argument("--lang=en-US")

# ban the picture and css for faster speed
#prefs = {"profile.managed_default_content_settings.images": 2, 'permissions.default.stylesheet': 2}
#options.add_experimental_option("prefs", prefs)

# dirver initialization
# remember the geck0 driver location
# C:\Users\V530\Downloads\chromedriver.exe"
driver = webdriver.Firefox(service=Service(r'C:\Program Files (x86)\geckodriver.exe'),options=options)
driver.set_page_load_timeout(10)

#### 3. Define the weblist you want to scrape



In [14]:
#folder to store the scraped reviews
product_folder = 'C:/Users/ASUS/Desktop/NLP/Assignment 1/reviews'
brand = 'Seagate '
model = 'STKB2000400'

web_dict = {
    'Seagate - One Touch 2TB External USB 3.0 Portable Hard Drive with Rescue Data Recovery Services - Black':"https://www.bestbuy.com/site/reviews/seagate-one-touch-2tb-external-usb-3-0-portable-hard-drive-with-rescue-data-recovery-services-black/6439172?variant=A&sort=MOST_RECENT"
}

# define the maxinum page for reviews or you need (try exception module)
web_pages = {
    'Seagate - One Touch 2TB External USB 3.0 Portable Hard Drive with Rescue Data Recovery Services - Black': 21   #change this number to the "(total reviews/20) + 1" you want to scrape, i.e. if there are 3000 reviews to scrape, then this value is (3000/20) + 1= 151
}

# A page have about 10 reviews, sometimes i face problems
    # you for 6460 reviews, set 646
        
    # for 4590 reviews, set 459 page



In [15]:
# Define the data to scrape
css_path_reviews = "[class='review-item-content col-xs-12 col-md-9']"
css_path_review_text = "[class='ugc-review-body'] > p"
css_path_review_date = "[class='posted-date-ownership disclaimer v-m-right-xxs'] > [class='submission-date']"

# Initialize the Firefox WebDriver
driver = webdriver.Firefox(service=Service(r'C:\Program Files (x86)\geckodriver.exe'), options=options)
driver.set_page_load_timeout(30)

# Scrape the reviews
for name, url in web_dict.items():
    reviews_list = []
    for i in range(1, web_pages[name] + 1):  # Adjust range to include the last page
        try:
            driver.get(url + str(i))
            logging.info(f"Page {i}: {driver.current_url}")
            WebDriverWait(driver, 200).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_path_reviews)))
            review_elements = driver.find_elements(By.CSS_SELECTOR, css_path_reviews)
            for j, element in enumerate(review_elements, start=1):
                try:
                    child_element_text = element.find_element(By.CSS_SELECTOR, css_path_review_text)
                    child_element_date = element.find_element(By.CSS_SELECTOR, css_path_review_date)
                    date = child_element_date.get_attribute("title")
                    reviews_list.append({'brand': brand, 'model': model, 'product': name, 'date': date, 'review': child_element_text.text})
                    logging.info(f"Review {j} on page {i} extracted")
                except NoSuchElementException:
                    logging.error(f"Error extracting review {j} on page {i}: element not found")
        except TimeoutException:
            logging.error(f"Page {i} timed out")
    # Save the DataFrame to the file path
    file_path = os.path.join(product_folder, name.replace('/', '-').replace('"', '-') + ".csv")
    pd.DataFrame(reviews_list).to_csv(file_path, index=False)
    logging.info(f"Reviews saved to {file_path}")

# Close the WebDriver session
driver.quit()